<a href="https://colab.research.google.com/github/victordaniel/Deeplearning/blob/master/PCB_Alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [1]:
%tensorflow_version 1.x


TensorFlow 1.x selected.


In [0]:
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import Sequential


In [3]:

from matplotlib import pyplot
%matplotlib inline
#from scipy.misc import toimage
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers import BatchNormalization
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from sklearn.model_selection import train_test_split
#K.set_image_dim_ordering('tf')


Using TensorFlow backend.


In [4]:
%tensorflow_version

Currently selected TF version: 1.x
Available versions:
* 1.x
* 2.x


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
import os
print(os.getcwd())
print(os.chdir('/content/drive/My Drive/PCB/Classification/Aug_pcb'))
print(os.listdir())

/content
None
['test.py', 'x_train.npy', 'y_train.npy', 'PCB_ALEXNET.h5', 'PCB_ALEXNET.h5.csv']


In [0]:
from IPython.display import Image,display
#display(Image(filename='/content/drive/My Drive/SSDL/Day2/Image/alex.png',embed=True))

In [9]:
import numpy as np
data_x=np.load('x_train.npy')
data_y=np.load('y_train.npy')
print(len(data_x))
print(len(data_y))
train_x=data_x
train_y=data_y
print(train_x.shape)
print(train_y.shape)
print(np.unique(train_y))
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2)


print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

5040
5040
(5040, 300, 300, 3)
(5040,)
[0. 1.]
(4032, 300, 300, 3)
(4032,)
(1008, 300, 300, 3)
(1008,)


In [10]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

4032
4032
1008
1008


In [11]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4032, 300, 300, 3)
(4032,)
(1008, 300, 300, 3)
(1008,)


**Pre-Processing**

In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [0]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [0]:
from keras.optimizers import adam
from keras import regularizers

**AlexNet Model**

In [0]:
def AlexNet_model():

    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), input_shape=(300, 300, 3), activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    
    model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    
    model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))
    model.add(Flatten())
    
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    
    model.add(Dense(num_classes, activation='sigmoid'))
    
    lrate = 0.0001
    epochs=13
    decay = lrate/epochs
    optimizer =adam(lr=0.0001)
    #sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [0]:
#os.chdir('/content/drive/My Drive/Colab Notebooks/DL_Practical_MLP/Results/MLP_CIFAR10')
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping,ModelCheckpoint
mc = ModelCheckpoint('PCB_ALEXNET.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
filepath = "saved-model-{epoch:02d}.h5"
mc1 = ModelCheckpoint(filepath, monitor='val_loss', mode='min', verbose=1, save_best_only=True  ,period=10)
cv = keras.callbacks.CSVLogger('PCB_ALEXNET.h5.csv', separator=',', append=False)
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=75,  mode='auto')


In [54]:

"""
import os
os.chdir("/content/drive/My Drive/PCB")
from keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger
mc = ModelCheckpoint('cipharAlexNetNet.h5', monitor='val_loss', save_best_only=True)
tb = TensorBoard(log_dir='/content/cnn/My Drive/logs', write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, update_freq='epoch')
cv = CSVLogger('cipharAlexNetNet.csv',append=True)
"""

'\nimport os\nos.chdir("/content/drive/My Drive/PCB")\nfrom keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger\nmc = ModelCheckpoint(\'cipharAlexNetNet.h5\', monitor=\'val_loss\', save_best_only=True)\ntb = TensorBoard(log_dir=\'/content/cnn/My Drive/logs\', write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, update_freq=\'epoch\')\ncv = CSVLogger(\'cipharAlexNetNet.csv\',append=True)\n'

In [55]:
from keras.callbacks import ModelCheckpoint
"""
model = Sequential()
model.add(Dense(10, input_dim=784, kernel_initializer='uniform'))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
"""
'''
saves the model weights after each epoch if the validation loss decreased
'''
#checkpointer = ModelCheckpoint(filepath='/tmp/weights.hdf5', verbose=1, save_best_only=True)
#model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=0, validation_data=(X_test, Y_test), callbacks=[checkpointer])



'\nsaves the model weights after each epoch if the validation loss decreased\n'

In [0]:
from keras.losses import mean_squared_error,mean_absolute_error,mean_squared_logarithmic_error,categorical_hinge,hinge,squared_hinge,categorical_crossentropy
from keras.optimizers import adam,adadelta,adagrad,adamax,sgd
#model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])


In [57]:
model=AlexNet_model()
print(model.summary())


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 150, 150, 32)      9248      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 37, 37, 128)      

In [0]:
class WeightCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch%1 == 0:
            fig, axs = plt.subplots(1,10, figsize=(10*1.5,1.5), facecolor='w', edgecolor='k')
            for i in range(10):
                axs[i].set_title("Digit : " + str(i))
                digit = model.layers[0].get_weights()[0][:,i]
                digit = ((digit - digit.min())*(1/(digit.max() - digit.min()))*255).astype("uint8")
                axs[i].imshow(digit.reshape(300,300,3),cmap="viridis")
weight = WeightCallback()

In [0]:
checkpointer = ModelCheckpoint(filepath='/tmp/weights.hdf5', verbose=1, save_best_only=True)

**Keras Callbacks**

In [60]:
history=model.fit(X_train, y_train, validation_split=(0.2), epochs=20, batch_size=32,verbose=2,callbacks=[checkpointer])
#model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=0, validation_data=(X_test, Y_test), callbacks=[checkpointer])




Train on 3225 samples, validate on 807 samples
Epoch 1/20
 - 7s - loss: 0.6933 - acc: 0.4998 - val_loss: 0.6932 - val_acc: 0.4895

Epoch 00001: val_loss improved from inf to 0.69316, saving model to /tmp/weights.hdf5
Epoch 2/20
 - 6s - loss: 0.6679 - acc: 0.6009 - val_loss: 0.5679 - val_acc: 0.6964

Epoch 00002: val_loss improved from 0.69316 to 0.56794, saving model to /tmp/weights.hdf5
Epoch 3/20
 - 6s - loss: 0.4949 - acc: 0.7678 - val_loss: 0.4850 - val_acc: 0.7869

Epoch 00003: val_loss improved from 0.56794 to 0.48499, saving model to /tmp/weights.hdf5
Epoch 4/20
 - 6s - loss: 0.3651 - acc: 0.8493 - val_loss: 0.3627 - val_acc: 0.8414

Epoch 00004: val_loss improved from 0.48499 to 0.36274, saving model to /tmp/weights.hdf5
Epoch 5/20
 - 6s - loss: 0.2862 - acc: 0.8884 - val_loss: 0.2943 - val_acc: 0.8711

Epoch 00005: val_loss improved from 0.36274 to 0.29428, saving model to /tmp/weights.hdf5
Epoch 6/20
 - 6s - loss: 0.2267 - acc: 0.9126 - val_loss: 0.2428 - val_acc: 0.8897

Epo

In [61]:
model.load_weights('/tmp/weights.hdf5')
test_loss,test_acc=model.evaluate(X_test,y_test)
print(test_loss)
print(test_acc)

1008/1008 [==============================] - 1s 945us/step
0.2938901672790211
0.9365079365079365


### Loss and Accuracy Curves

In [62]:
train_loss = history.history['loss']
train_acc = history.history['acc']
val_loss = history.history['val_loss']
val_acc = history.history['val_acc']
xc=np.range(12)

AttributeError: ignored

In [63]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(xc, train_loss)
plt.plot(xc, val_loss)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(['train','val'])

NameError: ignored

In [64]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(xc, train_acc)
plt.plot(xc, val_acc)
plt.xlabel("Epoch")
plt.ylabel("Acc")
plt.legend(['train','val'])

NameError: ignored

In [65]:
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix
import itertools

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)
target_names = ['class 0', 'class 1']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.90      0.98      0.94       498
     class 1       0.98      0.90      0.93       510

    accuracy                           0.94      1008
   macro avg       0.94      0.94      0.94      1008
weighted avg       0.94      0.94      0.94      1008

